# Datensätze erfassen in Datenbank _NormalisierungsDemo_

In [3]:
USE NormalisierungsDemo;

Commands completed successfully.

Total execution time: 00:00:00.001

## INSERT-Anweisungen

Die Tabellen **Personal** und **Arbeitet\_An** werden _zuletzt_ befüllt, da ihre **Fremdschlüssel** sich auf Werte in anderen Tabellen beziehen.

In [4]:
-- Abteilung
INSERT INTO Abteilung (AbtNr, Name)
VALUES (1, 'Personal'),
    (2, 'Einkauf'),
    (3, 'Verkauf');

(3 rows affected)

Total execution time: 00:00:00.004

In [5]:
-- Projekt
INSERT INTO Projekt (ProjNr, Beschreibung)
VALUES (1, 'Kundenumfrage'),
    (2, 'Verkaufsmesse'),
    (3, 'Konkurrenzanalyse');

(3 rows affected)

Total execution time: 00:00:00.008

In [6]:
-- Tätigkeit
INSERT INTO Tätigkeit (TätigkeitsNr, Tätigkeit, Stundenlohn)
VALUES (1, 'Leitung', 50),
    (2, 'Bearbeitung', 30),
    (3, 'Präsentationsvorbereitung', 35);

(3 rows affected)

Total execution time: 00:00:00.008

In [7]:
-- Personal
INSERT INTO Personal (PersonalNr, Name, Vorname, AbtNr)
VALUES (1, 'Eichenau', 'Maria', 1),
    (2, 'Glahn', 'Stefanie', 2),
    (3, 'Kirsch', 'Karin', 2),
    (4, 'Conolly', 'Sean', 3); --,
   -- (5, 'Frawley', 'Lutz', 3); in Transaktionsbeispiel enthalten

(4 rows affected)

Total execution time: 00:00:00.009

In [8]:
-- Arbeitet_An
INSERT INTO Arbeitet_An (PersonalNr, ProjNr, TätigkeitsNr, Stunden)
VALUES (2, 1, 1, 25),
    (3, 1, 2, 55),
    (4, 1, 2, 70),
    (4, 2, 1, 25),
   -- (5, 2, 3, 160), in Transaktionsbeispiel enthalten
    (4, 3, 1, 25),
    (2, 3, 2, 80),
    (3, 3, 2, 65);


(7 rows affected)

Total execution time: 00:00:00.017

## Transaktionen

In [9]:
-- Beispiel einer erfolgreichen Transaktion 
-- Mitarbeiter Lutz Frawley wird angelegt und einem Projekt zugeordnet
USE NormalisierungsDemo;
BEGIN
    BEGIN TRANSACTION; -- Transaktion starten
    BEGIN TRY
        INSERT INTO Personal (PersonalNr, Name, Vorname, AbtNr)
            VALUES(5, 'Frawley', 'Lutz', 3);
        INSERT INTO Arbeitet_An (PersonalNr, ProjNr, TätigkeitsNr, Stunden)
            VALUES (5, 2, 3, 160);
        COMMIT TRANSACTION; -- Transaktion bestätigen
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION; -- Transaktion annullieren
        THROW; -- Fehlermeldung anzeigen
    END CATCH;
END;

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.004

In [10]:
-- Beispiel einer fehlerhaften Transaktion
-- Mitarbeiter Jane Doe wird angelegt und einem Projekt zugeordnet
USE NormalisierungsDemo;
BEGIN
    BEGIN TRANSACTION; -- Transaktion starten
    BEGIN TRY
        INSERT INTO Personal (PersonalNr, Name, Vorname, AbtNr)
            VALUES(6, 'Doe', 'Jane', 3);
        INSERT INTO Arbeitet_An (PersonalNr, ProjNr, TätigkeitsNr, Stunden)
            VALUES (6, 2, 99, 160); -- ungültige Tätigkeitsnummer!
        COMMIT TRANSACTION; -- Transaktion bestätigen
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION; -- Transaktion annullieren
        THROW; -- Fehlermeldung anzeigen
    END CATCH;
END;

(1 row affected)

(0 rows affected)

: Msg 547, Level 16, State 0, Line 9
Die INSERT-Anweisung steht in Konflikt mit der FOREIGN KEY-Einschränkung "fk_ArbeitetAn_Tätigkeit". Der Konflikt trat in der NormalisierungsDemo-Datenbank, Tabelle "dbo.Tätigkeit", column 'TätigkeitsNr' auf.

Total execution time: 00:00:00.006

In [11]:
-- Jane Doe wurde nicht angelegt!
SELECT PersonalNr, Name, Vorname FROM Personal;

(5 rows affected)

Total execution time: 00:00:00.108

PersonalNr,Name,Vorname
1,Eichenau,Maria
2,Glahn,Stefanie
3,Kirsch,Karin
4,Conolly,Sean
5,Frawley,Lutz
